# Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

Kruskal format is a decomposition of a tensor $\mathcal{X}$ as the sum of the outer products as the columns of matrices. For example, we might write:

$
\mathcal{X} = \sum_{r} a_r \circ b_r \circ c_r
$

where a subscript denotes column index and a circle denotes outer product. In other words, the tensor X is built from the columns of the matrices A,B, and C. It's often helpful to explicitly specify a weight for each outer product, which we do here:

$
\mathcal{X} = \sum_{r} \lambda_r \, a_r \circ b_r \circ c_r
$

The **`ktensor`** class stores the components of the tensor $\mathcal{X}$ and can perform many operations, e.g., **`ttm`**, without explicitly forming the tensor $\mathcal{X}$.

## Contents

* [Kruskal tensor format via ktensor](#Kruskal-tensor-format-via-ktensor)
* [Specifying weights in a ktensor](#Specifying-weights-in-a-ktensor)
* [Creating a one-dimensional ktensor](#Creating-a-one-dimensional-ktensor)
* [Constituent parts of a ktensor](#Constituent-parts-of-a-ktensor)
* [Creating a ktensor from its constituent parts](#Creating-a-ktensor-from-its-constituent-parts)
* [Creating an empty ktensor](#Creating-an-empty-ktensor)
* [Use full or tensor to convert a ktensor to a tensor
](#Use-full-or-tensor-to-convert-a-ktensor-to-a-tensor)
* [Use double to convert a ktensor to a multidimensional array](#Use-double-to-convert-a-ktensor-to-a-multidimensional-array)
* [Use tendiag or sptendiag to convert a ktensor to a ttensor](#Use-tendiag-or-sptendiag-to-convert-a-ktensor-to-a-ttensor)
* [____Break HERE____](#____Break-HERE____)
* [Creating an empty tensor](#Creating-an-empty-tensor)
* [Create a tensor of all zeroes](#Create-a-tensor-of-all-zeroes)
* [Convert tensor to an array](#Convert-tensor-to-an-array)
* [Use len, ndims, and size to get the size of a tensor](#Use-len,-ndims,-and-size-to-get-the-size-of-a-tensor)
* [Subscripted reference of a tensor](#Subscripted-reference-of-a-tensor)
* [Subscripted assignment for a tensor](#Subscripted-assignment-for-a-tensor)
* [Basic operations on a tensor](#Basic-operations-on-a-tensor)
     * [Addition](#Addition)
     * [Subtraction](#Subtraction)
     * [Multiplication](#Multiplication)
     * [Division](#Division)
     * [Power](#Power)
     * [Equality](#Equality)
* [Use permute to reorder the modes of a tensor](#Use-permute-to-reorder-the-modes-of-a-tensor)
* [Display tensor](#Display-tensor)

## Kruskal tensor format via ktensor

In [1]:
import pyttb as ttb
import numpy as np

Kruskal format stores a tensor as a sum of rank-1 outer products. For example, consider a tensor of the following form:

$
X = a_1 \circ b_1 \circ c_1 + a_2 \circ b_2 \circ c_2
$

This can be stored in Kruskal form as follows.

In [2]:
np.random.seed(0)
A = np.random.rand(4,2) # First column is a_1, second is a_2.
B = np.random.rand(3,2) # Likewise for B.
C = np.random.rand(2,2) # Likewise for C.
X = ttb.ktensor.from_factor_matrices([A,B,C]) # Create the ktensor.
X

ktensor of shape 4 x 3 x 2
weights=[1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

For Kruskal format, there can be any number of matrices, but every matrix must have the same number of columns. The number of rows can vary.

In [3]:
Y = ttb.ktensor.from_factor_matrices(
    [np.random.rand(4,1), np.random.rand(2,1), np.random.rand(3,1)]
) # Another ktensor.
Y

ktensor of shape 4 x 2 x 3
weights=[1.]
factor_matrices[0] =
[[0.77815675]
 [0.87001215]
 [0.97861834]
 [0.79915856]]
factor_matrices[1] =
[[0.46147936]
 [0.78052918]]
factor_matrices[2] =
[[0.11827443]
 [0.63992102]
 [0.14335329]]

## Specifying weights in a ktensor

Weights for each rank-1 tensor can be specified by passing in a column vector. For example: 

$
X = \lambda_1 \, a_1 \circ b_1 \circ c_1 + \lambda_2 \, a_2 \circ b_2 \circ c_2 $

## **TODO** : Propose only using the instantiation of X once here, since X is not modified going forward until section ***Subscripted assignment for a ktensor***, and it would save space in many of the subsequent code blocks without confusing the user.

In [4]:
np.random.seed(0)
A = np.random.rand(4,2) # Create some data.
B = np.random.rand(3,2)
C = np.random.rand(2,2)
weights = np.array([5.0, 0.25])
X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.
X

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## Creating a one-dimensional ktensor

In [5]:
np.random.seed(0)
Y = ttb.ktensor.from_factor_matrices(np.random.rand(4,5)) # A one-dimensional ktensor.
Y

ktensor of shape 4
weights=[1. 1. 1. 1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]]

## Constituent parts of a ktensor

In [6]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.weights # Weights or multipliers.

array([5.  , 0.25])

In [7]:
X.factor_matrices # Cell array of matrices.

[array([[0.5488135 , 0.71518937],
        [0.60276338, 0.54488318],
        [0.4236548 , 0.64589411],
        [0.43758721, 0.891773  ]]),
 array([[0.96366276, 0.38344152],
        [0.79172504, 0.52889492],
        [0.56804456, 0.92559664]]),
 array([[0.07103606, 0.0871293 ],
        [0.0202184 , 0.83261985]])]

## Creating a ktensor from its constituent parts

In [8]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

Y = ttb.ktensor.from_data(X.weights, X.factor_matrices) # Recreate X.
Y

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## Creating an empty ktensor

In [9]:
Z = ttb.ktensor() # Empty ktensor.
Z

ktensor of shape 
weights=[]

## Use full or tensor to convert a ktensor to a tensor

In [10]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.full() # Converts to a tensor.

tensor of shape 4 x 3 x 2
data[0, :, :] = 
[[0.19381804 0.11054766]
 [0.16256856 0.12266212]
 [0.12514704 0.16930925]]
data[1, :, :] = 
[[0.21086124 0.10221034]
 [0.17577751 0.10823074]
 [0.13259822 0.13959484]]
data[2, :, :] = 
[[0.1504007  0.09282405]
 [0.12657497 0.10501592]
 [0.09849813 0.1487711 ]]
data[3, :, :] = 
[[0.15722304 0.11380622]
 [0.13332549 0.13320036]
 [0.10626643 0.19694404]]

In [11]:
ttb.tensor.from_tensor_type(X) # Same as above.

tensor of shape 4 x 3 x 2
data[0, :, :] = 
[[0.19381804 0.11054766]
 [0.16256856 0.12266212]
 [0.12514704 0.16930925]]
data[1, :, :] = 
[[0.21086124 0.10221034]
 [0.17577751 0.10823074]
 [0.13259822 0.13959484]]
data[2, :, :] = 
[[0.1504007  0.09282405]
 [0.12657497 0.10501592]
 [0.09849813 0.1487711 ]]
data[3, :, :] = 
[[0.15722304 0.11380622]
 [0.13332549 0.13320036]
 [0.10626643 0.19694404]]

## Use double to convert a ktensor to a multidimensional array

In [12]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.double() # Converts to an array.

array([[[0.19381804, 0.11054766],
        [0.16256856, 0.12266212],
        [0.12514704, 0.16930925]],

       [[0.21086124, 0.10221034],
        [0.17577751, 0.10823074],
        [0.13259822, 0.13959484]],

       [[0.1504007 , 0.09282405],
        [0.12657497, 0.10501592],
        [0.09849813, 0.1487711 ]],

       [[0.15722304, 0.11380622],
        [0.13332549, 0.13320036],
        [0.10626643, 0.19694404]]])

## Use tendiag or sptendiag to convert a ktensor to a ttensor

A ktensor can be regarded as a ttensor with a diagonal core.

## TODO: tendiag() is bugging for me. [See issue]()

In [13]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

R = len(X.weights) # Number of factors in X.
# core = ttb.tendiag(X.weights, ((R, ) * X.ndims)) # Create a diagonal core.
# Y = ttb.ttensor.from_data(core, X.factor_matrices)
# Y

In [14]:
X.norm() # - Y.norm() # They are the same.

0.6953902774728465

# ____Break HERE____

In [15]:
len(X.weights)

2

In [16]:
X.ndims

3